In [65]:
import pandas as pd
import numpy as np
from datetime import datetime

In [66]:
#drug-drug_similarity
dr_str = pd.read_table("/DAS_Storage1/aschoi/data/Drug_Repositioning/5_new_drug/new_predict/1_drug_drug_str_sim_mat.tsv", index_col = 0)
dr_se = pd.read_table("/DAS_Storage1/aschoi/data/Drug_Repositioning/5_new_drug/new_predict/1_drug_drug_se_sim_mat.tsv", index_col = 0)
#disease-disease similarity
di_sim = pd.read_table("/DAS_Storage1/aschoi/data/Drug_Repositioning/6_new_disease/new_predict/1_disease_disease_combined_sim.tsv")

In [68]:
print len(di_sim.drop_duplicates('Disease_ID1'))

233


In [69]:
dr_sim = 1 - (1-dr_se)*(1-dr_str)

In [70]:
dr_sim.to_csv("/DAS_Storage1/aschoi/data/Drug_Repositioning/5_new_drug/new_predict/1_drug_drug_combined_sim_mat.tsv", sep='\t')

In [71]:
asso = pd.read_table("/DAS_Storage1/aschoi/data/Drug_Repositioning/7_new_association/new_predict/1_gold_learning_asso.tsv")

In [39]:
#drug-disease similarity

In [138]:
di_dr_mat = pd.DataFrame(columns = dr_ids, index= di_ids) #di에 대한 dr의 distance보기
cnt = 0 
print 'start : ' + datetime.now().strftime('%Y-%m-%d %H:%M:%S')
for i in dr_ids:
    other_drugs_sim = dr_sim[dr_sim.index != i][i]# 검사할 drug를 제외한 다른 drug들
    for j in di_ids:
        target_disease = asso[(asso.disease_id == j)] # 검사할 disease
        other_drugs_di_sim = other_drugs_sim.filter(items = target_disease['drug_id'].tolist())
        sum_selected_drug = other_drugs_di_sim.sum() # association이 있는 것들의 합
        sum_all_drug = other_drugs_sim.sum() # similarity의 전체 합
        norm = sum_selected_drug / sum_all_drug #normalize한 값
        di_dr_mat[i].ix[j] = norm
    cnt = cnt + 1
    if cnt % 100 == 0:
        print 'cnt == {} : '.format(cnt) + datetime.now().strftime('%Y-%m-%d %H:%M:%S')
print 'end : ' + datetime.now().strftime('%Y-%m-%d %H:%M:%S')

start : 2017-04-19 01:09:09
cnt == 100 : 2017-04-19 01:14:33
cnt == 200 : 2017-04-19 01:19:08
cnt == 300 : 2017-04-19 01:23:26
cnt == 400 : 2017-04-19 01:27:45
end : 2017-04-19 01:29:13


In [139]:
#disease-drug similarity #normalize 한 버전
dr_di_mat = pd.DataFrame(columns = dr_ids, index= di_ids) #dr에 대한 di의 distance보기
cnt = 0 
print 'start : ' + datetime.now().strftime('%Y-%m-%d %H:%M:%S')
for i in di_ids:
    other_diseases_sim = di_sim[(di_sim.Disease_ID1 == i) & (di_sim.Disease_ID2 != i)] # 검사할 disease를 제외한 다른 disease들
    for j in dr_ids:
        target_drug = asso[(asso.drug_id ==j)] # 검사할 drug
        flags = other_diseases_sim['Disease_ID2'].isin(target_drug['disease_id']) # 검사할 drug와 다른 disease들의 association여부
        sum_selected_disease = other_diseases_sim[flags]['Combined'].sum() # association이 있는 것들의 합
        sum_all_disease = other_diseases_sim['Combined'].sum() # similarity의 전체 합
        norm = sum_selected_disease / sum_all_disease #normalize한 값
        dr_di_mat[j].ix[i] = norm
    cnt = cnt + 1
    if cnt % 100 == 0:
        print 'cnt == {} : '.format(cnt) + datetime.now().strftime('%Y-%m-%d %H:%M:%S')
print 'end : ' + datetime.now().strftime('%Y-%m-%d %H:%M:%S')

start : 2017-04-19 01:29:13
cnt == 100 : 2017-04-19 01:34:16
cnt == 200 : 2017-04-19 01:39:18
end : 2017-04-19 01:40:59


In [140]:
bef_distance = -(di_dr_mat+dr_di_mat)
distance_mat = pd.DataFrame(columns = bef_distance.columns.values, index = bef_distance.index.values)
cnt = 0 
print 'start : ' + datetime.now().strftime('%Y-%m-%d %H:%M:%S')
for i in bef_distance.columns.values:
    for j in bef_distance.index.values:
        a = bef_distance[i].ix[j]
        distance = np.exp(a)
        distance_mat[i].ix[j] = distance
    cnt = cnt + 1
    if cnt % 100 == 0:
        print 'cnt == {} : '.format(cnt) + datetime.now().strftime('%Y-%m-%d %H:%M:%S')
print 'end : ' + datetime.now().strftime('%Y-%m-%d %H:%M:%S')

start : 2017-04-19 01:40:59
cnt == 100 : 2017-04-19 01:41:06
cnt == 200 : 2017-04-19 01:41:13
cnt == 300 : 2017-04-19 01:41:21
cnt == 400 : 2017-04-19 01:41:28
end : 2017-04-19 01:41:30


In [141]:
print di_dr_mat.min().min() #drisease-drug sim의 최소값
print di_dr_mat.max().max() #disease-drug sim의 최대값
print dr_di_mat.min().min() #drug-disease sim의 최소값
print dr_di_mat.max().max() #drug-disease sim의 최소값
print (di_dr_mat + dr_di_mat).min().min() #위 두 결과를 더한 matrix의 최소값 (sim의 합의 최소값)
print (di_dr_mat + dr_di_mat).max().max() #위 두 결과를 더한 matrix의 최대값 (sim의 합의 최대값)
print bef_distance.min().min() #  - (sim의 합의 최대값)--> 최소값 --> 이 값을 exp에 넣으면 거리는 최소가 됨.
print bef_distance.max().max() #  - (sim의 합의 최소값)--> 최대값 --> 이 값을 exp에 넣으면 거리는 최대가 됨.
print np.exp(bef_distance.min().min()) #거리의 최소값 = 가까운 거리
print np.exp(bef_distance.max().max()) # 거리의 최대값 = 먼 거리

0.889090625749
1.0
0.848865265108
1.0
1.74576498185
2.0
-2.0
-1.74576498185
0.135335283237
0.174511439808


In [ ]:
distance_mat.to_csv("/DAS_Storage1/aschoi/data/Drug_Repositioning/7_new_association/PREDICT_data/4_2_drug_disease_norm_distance_predict.tsv", sep='\t')
distance_mat.to_csv("/home/share/aschoi/nas/users/asolchoi/data/Drug_Repositioning/7_new_association/PREDICT_data/4_2_drug_disease_norm_distance_predict.tsv", sep='\t')

In [115]:
# Distance histogram 그리기 (2017.3.27. 추가)
val = list()
for index, row in distance_mat.iterrows():
    for k in row:
        val.append(k)
print len(val) # drug (446) * disease(235)

101122
